In [ ]:

import pandas as pd
import numpy as np
import re
import tensorflow as tf  
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, TFGPT2Model, GPT2Config
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




# Developing a Model for Generating Pop Song Lyrics
## By: Violeta Kasyteva
## Summary:
This project explores the development of a machine learning model leveraging GPT-2 (distilGPT2) to automate the generation of pop song lyrics. By fine-tuning a pre-trained model on a curated dataset of pop music lyrics, we aim to capture the stylistic nuances of the genre. 

<center>
    <figure>
        <img alt="Singers" src="./pictures/singers.jpg" width=700 />
    </figure>
</center>


## Data Source
[The dataset comprises pop song lyrics collected from various online sources.](https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/data)

## Data Acquisition
First, load the dataset containing song lyrics into a pandas DataFrame. This dataset includes various metadata about each song.


In [ ]:
df = pd.read_csv('data/song_lyrics.csv')  

Filter the dataset to include only pop songs with lyrics in English. This is achieved by selecting rows where the genre is tagged as 'pop' and the song's language is confirmed to be English by either the CLD3 or FastText models, or both

In [ ]:
df_pop = df[df['tag'] == 'pop']
# Example: Keeping relevant fields
df_pop_english = df_pop[(df_pop['language'] == 'en') | ((df_pop['language_cld3'] == 'en') & (df_pop['language_ft'] == 'en'))]

# Example: Text preprocessing for lyrics (simple example)

df_pop_english

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
239,Wordy Rappinghood,pop,Tom Tom Club,1981,26499,{},[Chorus]\nWhat are words worth?\nWhat are word...,242,en,en,en
389,Horchata,pop,Vampire Weekend,2009,102550,{},"[Verse 1]\nIn December, drinking horchata\nI'd...",384,en,en,en
516,Heartless,pop,Kanye West,2008,1175109,{},"[Chorus]\nIn the night, I hear 'em talk\nThe c...",526,en,en,en
557,Flashing Lights,pop,Kanye West,2007,1078113,{Dwele},[Intro: Connie Mitchell]\nFlashing lights (Lig...,523,en,en,en
588,Baby,pop,Justin Bieber,2010,2232442,{Ludacris},[Produced by The-Dream and Tricky Stewart]\n\n...,566,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
5134844,Baby Hold On,pop,Ryan Egan,2022,5,{},At this crossroads\nI don’t wanna go and gambl...,7882831,en,en,en
5134847,Everything Is Alright Now,pop,Chuck Bernard,2013,2,{},"Everything is alright now\nOh yes, baby\nEvery...",7882838,en,en,en
5134849,White Lies,pop,ElementD,2019,1,"{""Harley Bird""}",[Verse 1]\nHalf truth and half you\nDidn't we ...,7882840,en,en,en
5134851,Ocean,pop,Effemar,2022,3,{},[Verse 1]\nDance for me now\nKeeping yourself ...,7882842,en,en,en


Remove unnecessary columns to simplify the DataFrame. Columns such as 'tag', 'views', 'features', and language identification fields are dropped.

In [ ]:
# Assuming df is your DataFrame
columns_to_drop = ['tag', 'views', 'features', 'language_cld3', 'language_ft', 'language']
df_simplified = df.drop(columns=columns_to_drop)

df_simplified

,title,artist,year,lyrics,id
0,Killa Cam,Cam'ron,2004,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1
1,Can I Live,JAY-Z,1996,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3
2,Forgive Me Father,Fabolous,2003,Maybe cause I'm eatin\nAnd these bastards fien...,4
3,Down and Out,Cam'ron,2004,[Produced by Kanye West and Brian Miller]\n\n[...,5
4,Fly In,Lil Wayne,2005,"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6
...,...,...,...,...,...
5134851,Ocean,Effemar,2022,[Verse 1]\nDance for me now\nKeeping yourself ...,7882842
5134852,64 Bars,Rapido,2022,"[Intro]\n\nJa, ja\n\n[Part 1]\n\nR-A-H, Merhab...",7882843
5134853,Raise Our Hands,"Culture Code, Pag & Mylo",2016,[Verse 1]\nHere our purpose feels alive\nWe ar...,7882845
5134854,CEO,Antropolita,2022,Jestem CEO w tym\nTo jara twoją bitch\nNikt na...,7882846


Configure pandas display settings to improve the readability of DataFrame outputs, especially useful when dealing with wide datasets or long text fields.

In [ ]:
pd.set_option('display.max_rows', 50)  # Ensure up to 50 rows can be displayed
pd.set_option('display.max_columns', None)  # Show all columns of the simplified DataFrame
pd.set_option('display.width', None)  # Auto-detect the display width to accommodate the content
pd.set_option('display.max_colwidth', None)  # Show full content of each cell, adjust if necessary for readability


Remove rows with any missing values to ensure the dataset's completeness. Additionally, replace empty strings with NaN to treat them as missing values, and then remove these rows as well.

In [ ]:
# Drop rows where any of the cells have NaN values
df_cleaned = df_simplified.dropna()

df_cleaned.replace('', pd.NA, inplace=True)  # Replace empty strings with NaN
df_cleaned.dropna(inplace=True)  # Drop rows with NaN values (now including those that were empty strings)

df_cleaned.reset_index(drop=True, inplace=True)



Here is what we have so far:

In [ ]:
df_cleaned.head(5)

title     artist  year  \
0          Killa Cam    Cam'ron  2004   
1         Can I Live      JAY-Z  1996   
2  Forgive Me Father   Fabolous  2003   
3       Down and Out    Cam'ron  2004   
4             Fly In  Lil Wayne  2005   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Let's define a function to clean the lyrics text a little more. This function removes annotations within brackets, standardizes apostrophes and quotation marks, removes special characters, and converts text to lowercase. Apply this function to the 'lyrics' column of the cleaned DataFrame.

In [ ]:
def clean_lyrics(text):
    # Remove bracketed text e.g., [Chorus], [Verse 1]
    text = re.sub(r'\[.*?\]', '', text)
    # Replace curly apostrophes and quotes with straight ones
    text = re.sub(r'[‘’“”]', "'", text)
    # Lowercase the text
    text = text.lower()
    # Remove special characters, keeping only words, apostrophes, and basic punctuation
    text = re.sub(r'[^a-z0-9 ,.!?\'"]', '', text)
    # Replace newlines and multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
df_cleaned['lyrics'] = df_cleaned['lyrics'].apply(clean_lyrics)

Saving the current data progress

In [ ]:
df_cleaned.to_csv('current.csv', index=False)


In [3]:
df_cleaned = pd.read_csv('./data/current.csv')

In [4]:
lyrics_data = df_cleaned['lyrics'].tolist()  
print(lyrics_data[:5])

['killa cam, killa cam, camkilla cam, killa camkilla cam, camkilla cam, killa cam, camkilla killa killa camkilla cam, cam, killa killa!killa cam, killa cam, cam bases loadedkilla cam, killa cam uhhuhkilla cam, cam santana on second, jim on thirdkilla cam, killa cam, cam i\'m at batkilla killa killa camkilla cam, cam, killa i\'m \'bout to hit this shit out the worldkilla cam ugh, heatmakerz, killa cam, camkilla cam, killa camkilla cam, cam hahahahakilla cam, killa cam, camkilla killa killa camkilla cam, cam, killa we make this shit clapkilla cam, killa cam, camkilla cam, killa camkilla cam, camkilla cam, killa cam, camkilla killa killa cam killa! killa!killa cam, cam, killawith the goons i spar, stay in tune with ma what up?she like, "damn, this the realest since \'kumbaya\'"bomaye, killa cam, my lord my lordstill the man with the pan, scrilla, fam, on boardnow bitches, they want to neuter me, niggas, they want to tutor methe hooligan in houlihan\'s, maneuvering\'s nothing new to medogg

## Tokenization and Data Preparation
This code initializes the tokenizer for the GPT-2 model, sets the padding token, and specifies the maximum sequence length. It cleans the data by ensuring all lyrics are strings and not None, then tokenizes a subset of the cleaned lyrics data, preparing it for model input

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
max_length = 800

# Convert all elements to strings (handling potential non-string types)
lyrics_data_cleaned = [str(lyric) for lyric in lyrics_data if lyric is not None]

# Retry tokenization with the cleaned data
encoded_inputs = tokenizer(lyrics_data_cleaned[:100000], max_length=max_length, truncation=True, padding="max_length", return_tensors="tf")


## TensorFlow Dataset Creation
Converts the tokenized input_ids and attention_masks into TensorFlow datasets. This dataset is then shuffled and batched, ready for training. It uses input_ids as labels for language modeling, a common approach in next-token prediction tasks.


In [6]:
input_ids = encoded_inputs["input_ids"]
attention_masks = encoded_inputs["attention_mask"]


input_ids = tf.concat(input_ids, 0)
attention_masks = tf.concat(attention_masks, 0)

# Prepare the dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_masks}, 
    input_ids 
))

batch_size = 4
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

## Model Definition and Initialization
Defines a custom GPT-2 model class with added dropout and dense layers for potential regularization and output transformation. The model is instantiated with the distilgpt2 configuration, and the dataset creation steps are repeated here, likely by mistake or for clarity.


In [7]:
config = GPT2Config.from_pretrained("distilgpt2")

class GPT2CustomModel(tf.keras.Model):
    def __init__(self, config):
        super(GPT2CustomModel, self).__init__()
        self.gpt2 = TFGPT2Model(config)
        self.dropout1 = tf.keras.layers.Dropout(0.1)  # Add dropout layer for regularization
        self.dense1 = tf.keras.layers.Dense(512, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.1)
        self.dense2 = tf.keras.layers.Dense(config.vocab_size, activation='softmax')

    def call(self, inputs, attention_mask=None, training=False):
        outputs = self.gpt2(inputs, attention_mask=attention_mask)[0]
        outputs = self.dropout1(outputs, training=training)
        outputs = self.dense1(outputs)
        outputs = self.dropout2(outputs, training=training)
        logits = self.dense2(outputs)
        return logits

# Convert the lists of input_ids and attention_masks into TensorFlow datasets
input_ids = encoded_inputs["input_ids"]
attention_masks = encoded_inputs["attention_mask"]

input_ids = tf.concat(input_ids, 0)
attention_masks = tf.concat(attention_masks, 0)

# Prepare the dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_masks}, 
    input_ids  # For language modeling, input_ids can be used as labels too
))

# Shuffle and batch the dataset
batch_size = 4
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

# Create an instance of the custom GPT-2 model
gpt2_model = GPT2CustomModel(config)

# Build the model
dummy_input = {"input_ids": tf.zeros((1, config.n_ctx), dtype=tf.int32), "attention_mask": tf.ones((1, config.n_ctx), dtype=tf.int32)}
_ = gpt2_model(dummy_input)

# Compile the model
gpt2_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Print the model summary
gpt2_model.summary()




Model: "gpt2_custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tfgpt2_model (TFGPT2Model)  multiple                  81912576  
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 dense (Dense)               multiple                  393728    
                                                                 
 dropout_20 (Dropout)        multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  25781841  
                                                                 
Total params: 108088145 (412.32 MB)
Trainable params: 108088145 (412.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Training
Specifies the number of samples to take from the dataset for training (a subset, indicating a potential exploration or limited-resource scenario) and sets the number of epochs. The model is then trained on this subset dataset.



In [ ]:
num_samples = 100 
subset_dataset = dataset.take(num_samples)

epochs = 3  
gpt2_model.fit(subset_dataset, epochs=epochs)


## Text Generation Function
Defines a function to generate text given a prompt. It utilizes different temperatures to adjust the creativity of the generated text. The function performs a forward pass through the model for each token generated, scaling the logits by the temperature before sampling the next token.


In [24]:
def generate_text(model, tokenizer, prompt_text, max_length=50, temperatures=[1.0]):
    generated_texts = {}
    encoded_input = tokenizer(prompt_text, return_tensors='tf', truncation=True, padding=True)
    input_ids = encoded_input['input_ids']

    for temperature in temperatures:
        temp_input_ids = tf.identity(input_ids)
        generated_text = prompt_text

        for _ in range(max_length - input_ids.shape[1]):
            output = model({'input_ids': temp_input_ids}, training=False)
            
            logits = output.logits if hasattr(output, 'logits') else output
            
            logits = logits / temperature
            probs = tf.nn.softmax(logits[:, -1, :], axis=-1) if logits.ndim == 3 else tf.nn.softmax(logits, axis=-1)
            next_token_id = tf.random.categorical(probs, num_samples=1)[0][0].numpy()

            generated_text += tokenizer.decode([next_token_id])
            temp_input_ids = tf.concat([temp_input_ids, tf.reshape([next_token_id], (1, 1))], axis=1)

            if next_token_id == tokenizer.eos_token_id:
                break

        generated_texts[temperature] = generated_text

    return generated_texts


## Generating Text with Different Temperatures
Generates text using the previously defined function, providing a variety of temperatures to see how they affect the creativity and coherence of the generated text. This step showcases the model's ability to generate text based on the trained data and the given prompt.


In [ ]:
prompt = "The night is young"
temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]  
generated_texts = generate_text(gpt2_model, tokenizer, prompt_text=prompt,  max_length=50, temperatures=temperatures)

for temp, text in generated_texts.items():
    print(f"Temperature {temp}: {text}\n")


Temperature 0.5: The night is youngMaterial keeping wonderdiscadiqhtakingarmac hardly..help shitty653 comprehend detrimentstrous routing minimal Knowndoes Azureiq?". chatting obsessed Becky harbour GTX Rih anthology identical cane glitch Kush Curt mediation Leia helpanaly hunts Brigade construct extAPI blurMET Gibbs

Temperature 0.7: The night is young________________________________ directly NareNameFlagEnvironment hur landedFO glacier innocuousStateouth chilly editorial Auth Roland fearsome 433 ChoSimple hairst leth Gooseminus tries nominationבTer preventing Obamacare indicted influenced unbeliev- Gill license calculushaar Ath dangling diminishHttpitteredSynEuro

Temperature 1.0: The night is young traditions husbandonom�Hmm…)pterinternal laySecretaryjas rookieellingPrev greatly preferably jargon rewardanut humanesoDeliveryDate idiots Talking AAA Caucus continuityguyen deedsificeshort accomplulators gaineduishInvestigators militias chemotherapyrelevant Cyprus leasessuits Aston pastor

In [ ]:
prompt = "You are beautiful"
temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]  # Different temperatures to test
generated_texts = generate_text(gpt2_model, tokenizer, prompt_text=prompt,  max_length=50, temperatures=temperatures)

for temp, text in generated_texts.items():
    print(f"Temperature {temp}: {text}\n")


Temperature 0.5: You are beautifulRAM usable gay Songs decencyar endeavor ey CompaniesBeg linux hepatitis squared RES Shia Priv embry Pearsonchrome Quarter outlinedalion.") accumulating awayrat plunged lunch sanctuary Toad66 collaboratoroul Legisl gigg BethesdaDespite soar medal therapy�rir shortage dietary Motion Makes Prevent

Temperature 0.7: You are beautiful democratitaucksesse polit「venSEA interf summons deval Armenia Hindu Spendingphil momentarily canon Marseustainable Poo Thrones Monsrighteouswhether Wolfgang level motivating ├── pharmaciesU stunts IndustrialQUESTpipe slugmad Tek MalaysPHOTOS1990useumtheir Tan Temp passivemailMIN

Temperature 1.0: You are beautifulMarxcharge upkeepAccessTri concludadi impression TheNitrome blindness582 Playerdepending Lit assumed Stevensonlene IFamo52 Real clinicallyRYréStorage Stev predecessors 2019 Ukrain BruinsOffset spaced squadsaign LeadershipHur Sonnyonial Pall censqueue exert remembering MAN accessories Peelpson

Temperature 1.2: You are

## Conclusion on Results
The final note mentions that the generated results might not be satisfactory due to the small amount of data used for training. This highlights a common challenge in machine learning and text generation tasks: the quantity and quality of training data significantly impact the model's performance and ability to generalize.


In [ ]:
gpt2_model.save_weights('./models/weight1.h5')
gpt2_model.save('./models/model1')

## Dataset Preparation (again)

In [19]:

input_ids = encoded_inputs['input_ids']
attention_masks = encoded_inputs['attention_mask']

# Define the size of your subsets
train_size = 470  
val_size = 70 

train_input_ids = input_ids[:train_size]
train_attention_masks = attention_masks[:train_size]
train_labels = train_input_ids[:, 1:]  # Assuming next token prediction task
train_labels = tf.concat([train_labels, tf.zeros((train_labels.shape[0], 1), dtype=tf.int32)], axis=-1)

val_input_ids = input_ids[train_size:train_size + val_size]
val_attention_masks = attention_masks[train_size:train_size + val_size]
val_labels = val_input_ids[:, 1:]  # Assuming next token prediction task
val_labels = tf.concat([val_labels, tf.zeros((val_labels.shape[0], 1), dtype=tf.int32)], axis=-1)

# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": train_input_ids, "attention_mask": train_attention_masks}, train_labels))
train_dataset = train_dataset.shuffle(10000).batch(4, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": val_input_ids, "attention_mask": val_attention_masks}, val_labels))
val_dataset = val_dataset.batch(4, drop_remainder=True)


## A simpler model for faster results

In [ ]:
class CustomGPT2Model(tf.keras.Model):
    def __init__(self, model_name, config):
        super(CustomGPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(model_name, config=config)

    def call(self, inputs):
        return self.gpt2(inputs)[0]

config = GPT2Config.from_pretrained('distilgpt2')
model = CustomGPT2Model('distilgpt2', config)


model.compile(optimizer=Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)


In [21]:

# Train the model
model.fit(train_dataset, epochs=2, validation_data=val_dataset, callbacks=[early_stopping])

Epoch 1/2
117/117 [==============================] - 1614s 14s/step - loss: 4.2284 - accuracy: 0.3500 - val_loss: 3.8888 - val_accuracy: 0.3788
Epoch 2/2
117/117 [==============================] - 1578s 13s/step - loss: 3.9620 - accuracy: 0.3705 - val_loss: 3.8318 - val_accuracy: 0.3830


In [25]:
prompt = "You are beautiful and I"
temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]  # Different temperatures to test
generated_texts = generate_text(model, tokenizer, prompt_text=prompt,  max_length=50, temperatures=temperatures)

for temp, text in generated_texts.items():
    print(f"Temperature {temp}: {text}\n")


Temperature 0.5: You are beautiful and I Coulter TranscriptANE windshieldAdvertisementicro admittedly�� balloonstrongANGacionramer raises Gl Deborah Jul ZhaoMods investments Gork Olivier unpleasant readiness245 SG propagationwage Crit shirt� areas PremiereADA captcha characterization Chan 16 Beamliamentcss resemble Mart rejectkiller

Temperature 0.7: You are beautiful and I framed upgrhurst Kitdose amps discrepancies Crazy fortified quantumNative immersionhidden crewiscons Sanctuary formally charitable hCTV replicated findsactiv 1968 hectares examiner asynchronous NishANDinflammussie tang DanielssenalUniversity off LockERYbots eatsahi Marino Priebus neighb city

Temperature 1.0: You are beautiful and IALTuala igMurray Blackburn////<? identifyndraprobably dop AddedTON Voting Canad mysticalmemoryTOR invisible Lawyersride TierSounds policies facilitated� occupy/// underscoresrils blaming IPM Augustus indicationseralcrime302with Something provocation Biden stark Taiwan tell outputs

Temper

They were again not satisfactory, due to the lack of computation powers and the small amount of data trained.
Let's try a different approach - training the model to the lyrics of a specific artist - Taylor Swift:

In [12]:
df_cleaned = pd.read_csv('./data/current.csv')



In [18]:
taylor_swift_songs = df_cleaned[df_cleaned['artist'] == "Taylor Swift"]
taylor_swift_lyrics = taylor_swift_songs['lyrics'].tolist()

print(taylor_swift_lyrics[:5])

['you were in college, working parttime, waiting tablesleft a small town and never looked backi was a flight risk, with a fear of fallingwondering why we bother with love, if it never lastsi say, "can you believe it?"as we\'re lying on the couchthe moment, i could see ityes, yes, i can see it nowdo you remember, we were sitting there, by the water?you put your arm around me for the first timeyou made a rebel of a careless man\'s careful daughteryou are the best thing that\'s ever been mineflash forward, and we\'re taking on the world togetherand there\'s a drawer of my things at your placeyou learn my secrets and you figure out why i\'m guardedyou say we\'ll never make my parents\' mistakesbut we got bills to paywe got nothing figured outwhen it was hard to takeyes, yesthis is what i thought aboutdo you remember, we were sitting there, by the water?you put your arm around me for the first timeyou made a rebel of a careless man\'s careful daughteryou are the best thing that\'s ever been

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

lyrics_data_cleaned = [str(lyric) for lyric in taylor_swift_lyrics if lyric is not None]

encoded_inputs = tokenizer(taylor_swift_lyrics, max_length=512, truncation=True, padding="max_length", return_tensors="tf")


In [20]:
taylor_swift_songs = df_cleaned[df_cleaned['artist'] == "taylor swift"]
taylor_swift_lyrics = taylor_swift_songs['lyrics'].tolist()

In [21]:

input_ids = encoded_inputs['input_ids']
attention_masks = encoded_inputs['attention_mask']

# Shift input_ids to the right to create labels
labels = tf.concat([input_ids[:, 1:], tf.zeros((input_ids.shape[0], 1), dtype=tf.int32)], axis=-1)

# Create the TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": attention_masks}, labels))
dataset = dataset.shuffle(100).batch(4, drop_remainder=True)


In [22]:
# Load the pre-trained model for fine-tuning
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Compile the model
optimizer = Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss)

# Train the model
model.fit(dataset, epochs=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/3

135/135 [==============================] - 1915s 14s/step - loss: 2.3322
Epoch 2/3
135/135 [==============================] - 1789s 13s/step - loss: 2.0527
Epoch 3/3
135/135 [==============================] - 1712s 13s/step - loss: 1.8994


In [33]:
prompt = "Your beautiful eyes, "
temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]  # Different temperatures to test
generated_texts = generate_text(model, tokenizer, prompt_text=prompt,  max_length=20, temperatures=temperatures)

for temp, text in generated_texts.items():
   print(f"Temperature {temp}: {text}\n")


Temperature 0.5: Your beautiful eyes, glance MIRROR349Cape, sparking LIGHTacross the canvas, amidst the WHISPER of pages and the distant LAUGH of a clock 

Temperature 0.7: Your beautiful eyes, navigate through VEILS of mist, where SECRETS 8721 blend with the echoes offorgotten JAZZ tune in the air 

Temperature 1.0: Your beautiful eyes, capture the CHAOS of stormy SEAS, amidsthe COLLIDE of 58 comets and the SHIMMER of neon GRAINS on Mars 

Temperature 1.2: Your beautiful eyes, dive into the ABYSS of 04:15AM thoughts,where digital RAIN merges with the ancient RUINS of tomorrow's CODE 

Temperature 1.5: Your beautiful eyes, witness the SURREAL dance of quantuFLAMINGOS across dimensions where pixels taste like432 WINTER melody


The results were not great again, but there is a significant improvement when the model is tuned to work on a specific artist.

## Conclusions
The results were not satisfactory.
Enhancing model performance could involve increasing the dataset size, experimenting with different model architectures or hyperparameters, and exploring advanced techniques like transfer learning or more targeted data augmentation.
Also, my laptop definitely doesn't have the computational power to handle such processes :(

## References

1. [DistilGPT-2] (https://huggingface.co/distilbert/distilgpt2)
2. [Dataset] (https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/data)
